In [1]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout
import os, string

In [2]:
sz = 128
# Step 1 - Building the CNN

# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))


# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
#units changed 128 to 400
classifier.add(Dense(units=400, activation='relu'))
classifier.add(Dropout(0.40))
#units changed 96 to 800
classifier.add(Dense(units=800, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=26, activation='softmax')) # softmax for more than 2

# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2


In [3]:
# Step 2 - Preparing the train/test data and training the model
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 28800)             0         
                                                                 
 dense (Dense)               (None, 400)               1

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rotation_range=30, # rotate the image 30 degrees
        width_shift_range=0.1, # Shift the pic width by a max of 10%
        height_shift_range=0.1, # Shift the pic height by a max of 10%
        rescale=1/255, # Rescale the image by normalzing it.
        shear_range=0.2, # Shear means cutting away part of the image (max 20%)
        zoom_range=0.2, # Zoom in by 20% max
        horizontal_flip=True, # Allo horizontal flipping
        fill_mode='nearest' # Fill in missing pixels with the nearest filled value
        )
test_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  fill_mode='nearest')

In [5]:
training_set = train_datagen.flow_from_directory('./ProcessedData/train',
                                                 target_size=(sz, sz),
                                                 batch_size=10,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('./ProcessedData/test',
                                            target_size=(sz , sz),
                                            batch_size=10,
                                            color_mode='grayscale',
                                            class_mode='categorical') 

Found 13000 images belonging to 26 classes.
Found 5200 images belonging to 26 classes.


In [6]:
training_set.class_indices

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [7]:
test_set.class_indices

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [8]:
classifier.fit(
        training_set,
        steps_per_epoch=449, 
        epochs=10,
        validation_data=test_set,
        validation_steps=190)

Epoch 1/10
449/449 [==============================] - 71s 157ms/step - loss: 2.2133 - accuracy: 0.3216 - val_loss: 0.9288 - val_accuracy: 0.7121
Epoch 2/10
449/449 [==============================] - 70s 155ms/step - loss: 0.8676 - accuracy: 0.7165 - val_loss: 0.4691 - val_accuracy: 0.8521
Epoch 3/10
449/449 [==============================] - 70s 155ms/step - loss: 0.5615 - accuracy: 0.8176 - val_loss: 0.3826 - val_accuracy: 0.8816
Epoch 4/10
449/449 [==============================] - 69s 153ms/step - loss: 0.4097 - accuracy: 0.8630 - val_loss: 0.2451 - val_accuracy: 0.9195
Epoch 5/10
449/449 [==============================] - 69s 154ms/step - loss: 0.3284 - accuracy: 0.8976 - val_loss: 0.2457 - val_accuracy: 0.9211
Epoch 6/10
449/449 [==============================] - 69s 154ms/step - loss: 0.3043 - accuracy: 0.9038 - val_loss: 0.2007 - val_accuracy: 0.9389
Epoch 7/10
449/449 [==============================] - 69s 154ms/step - loss: 0.2644 - accuracy: 0.9118 - val_loss: 0.1944 - val_ac

In [186]:
import numpy as np
from keras.preprocessing import image
test_file = './ProcessedData/Test/J/90.jpg'

In [187]:
import matplotlib.pyplot as plt
import cv2

In [188]:
test_img = image.load_img(test_file,target_size=(128, 128))

In [189]:
test_img = image.img_to_array(test_img)

In [190]:
test_img = cv2.cvtColor(np.array(test_img),cv2.COLOR_RGB2GRAY)

In [191]:
test_img.shape

(128, 128)

In [192]:
test_img = np.expand_dims(test_img, axis=0)
test_img = test_img/255

In [193]:
prediction_prob = classifier.predict(test_img)

In [194]:
prediction_prob

array([[3.0770412e-18, 1.5377906e-09, 1.6252721e-16, 2.1262241e-17,
        6.0253824e-14, 1.4252166e-16, 9.7562177e-22, 4.0637791e-18,
        2.9627875e-18, 1.0000000e+00, 7.5908945e-14, 4.3521851e-08,
        4.3753969e-09, 4.3914494e-09, 6.9203264e-16, 2.1222082e-14,
        5.3748894e-11, 1.4949746e-11, 2.0978647e-17, 7.2470747e-16,
        4.8569466e-15, 1.9685978e-13, 2.9278693e-17, 3.4860140e-10,
        2.5016305e-11, 4.8639676e-10]], dtype=float32)

In [195]:
np.argmax(prediction_prob)

9

In [196]:
classes = list(string.ascii_uppercase)

In [197]:
classes[np.argmax(prediction_prob)]

'J'

In [23]:
#saving the model
import os
if not os.path.exists("./Model"):
    os.mkdir("./Model")
classifier.save("./Model/Sign_classifier.h5")

In [ ]:
classifier_Json = classifier.to_json()
with open("./Model/classifier_json.json", "w") as json_file:
    json_file.write(classifier_Json)
print('json model saved')
classifier.save_weights('./Model/classifier_weights.h5')
print('Weights saved')

# Classifier Building - Completed